In [1]:
import os
os.environ["PYTHONNET_RUNTIME"] = "coreclr"
os.environ["DOTNET_SYSTEM_DRAWING_USE_GDIPLUS"] = "1"

import clr
import numpy as np
import time
from datetime import timedelta
from pythonnet import load
from System import Environment, Array, String

# Load .NET Core runtime
load("coreclr")

# Try to import System.IO; fallback to os.chdir if it fails
try:
    from System.IO import Directory, Path, File
    use_dotnet_dir = True
except Exception as e:
    print(f"Note: System.IO import failed ({e}), using os.chdir instead")
    import os
    use_dotnet_dir = False

# Define DWSIM path
dwsimpath = "/usr/local/lib/dwsim/"

# Set working directory
if use_dotnet_dir:
    Directory.SetCurrentDirectory(dwsimpath)
else:
    os.chdir(dwsimpath)

# Add DWSIM assemblies
clr.AddReference(dwsimpath + "CapeOpen.dll")
clr.AddReference(dwsimpath + "DWSIM.Automation.dll")
clr.AddReference(dwsimpath + "DWSIM.Interfaces.dll")
clr.AddReference(dwsimpath + "DWSIM.GlobalSettings.dll")
clr.AddReference(dwsimpath + "DWSIM.SharedClasses.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.dll")
clr.AddReference(dwsimpath + "DWSIM.UnitOperations.dll")
clr.AddReference(dwsimpath + "DWSIM.Inspector.dll")
clr.AddReference(dwsimpath + "System.Buffers.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.ThermoC.dll")

# Now import DWSIM types
from DWSIM.Interfaces.Enums.GraphicObjects import ObjectType
from DWSIM.Thermodynamics import Streams, PropertyPackages
from DWSIM.UnitOperations import UnitOperations
from DWSIM.Automation import Automation3
from DWSIM.GlobalSettings import Settings

print("DWSIM imports successful!")

DWSIM imports successful!


In [2]:
# Create an instance of the Automation3 class from the DWSIM.Automation module
# This class provides methods for automating tasks in DWSIM, such as creating and manipulating flowsheets
interf = Automation3()

In [3]:
# Creates a flowsheet
sim = interf.CreateFlowsheet()

In [4]:
# Add Compounds
sim.AddCompound("Water")

In [5]:
one = sim.AddFlowsheetObject('Material Stream','1')
two = sim.AddFlowsheetObject('Material Stream','2')
three = sim.AddFlowsheetObject('Material Stream','3')
four = sim.AddFlowsheetObject('Material Stream','4')
five = sim.AddFlowsheetObject('Material Stream','5')
six = sim.AddFlowsheetObject('Material Stream','6')
seven = sim.AddFlowsheetObject('Material Stream','7')
eight = sim.AddFlowsheetObject('Material Stream','8')
E1 = sim.AddFlowsheetObject('Energy Stream','E1')
E2 = sim.AddFlowsheetObject('Energy Stream','E2')
E3 = sim.AddFlowsheetObject('Energy Stream','E3')
E4 = sim.AddFlowsheetObject('Energy Stream','E4')
HT_1 = sim.AddFlowsheetObject('Heater','HT-1')
HT_2 = sim.AddFlowsheetObject('Heater','HT-2')
HT_3 = sim.AddFlowsheetObject('Heater','HT-3')
HT_4 = sim.AddFlowsheetObject('Heater','HT-4')

In [6]:
one = one.GetAsObject()
two = two.GetAsObject()
three = three.GetAsObject()
four = four.GetAsObject()
five = five.GetAsObject()
six = six.GetAsObject()
seven = seven.GetAsObject()
eight = eight.GetAsObject()
E1 = E1.GetAsObject()
E2 = E2.GetAsObject()
E3 = E3.GetAsObject()
E4 = E4.GetAsObject()
HT_1 = HT_1.GetAsObject()
HT_2 = HT_2.GetAsObject()
HT_3 = HT_3.GetAsObject()
HT_4 = HT_4.GetAsObject()

In [7]:
sim.ConnectObjects(one.GraphicObject , HT_1.GraphicObject, -1, -1)
sim.ConnectObjects(E1.GraphicObject , HT_1.GraphicObject, -1, -1)
sim.ConnectObjects(HT_1.GraphicObject , two.GraphicObject, -1, -1)
sim.ConnectObjects(three.GraphicObject , HT_2.GraphicObject, -1, -1)
sim.ConnectObjects(E2.GraphicObject , HT_2.GraphicObject, -1, -1)
sim.ConnectObjects(HT_2.GraphicObject , four.GraphicObject, -1, -1)
sim.ConnectObjects(five.GraphicObject , HT_3.GraphicObject, -1, -1)
sim.ConnectObjects(E3.GraphicObject , HT_3.GraphicObject, -1, -1)
sim.ConnectObjects(HT_3.GraphicObject , six.GraphicObject, -1, -1)
sim.ConnectObjects(seven.GraphicObject , HT_4.GraphicObject, -1, -1)
sim.ConnectObjects(E4.GraphicObject , HT_4.GraphicObject, -1, -1)
sim.ConnectObjects(HT_4.GraphicObject , eight.GraphicObject, -1, -1)

In [8]:
sim.AutoLayout()

In [9]:
one.SetOverallComposition(Array[float]([1.0]))
one.SetTemperature(300.0) # K
one.SetPressure(101325.0) # Pa
one.SetMassFlow(1.0) # kg/s

three.SetOverallComposition(Array[float]([1.0]))
three.SetTemperature(300.0) # K
three.SetPressure(101325.0) # Pa
three.SetMassFlow(1.0) # kg/s

five.SetOverallComposition(Array[float]([1.0]))
five.SetTemperature(300.0) # K
five.SetPressure(101325.0) # Pa
five.SetMassFlow(1.0) # kg/s

seven.SetOverallComposition(Array[float]([1.0]))
seven.SetTemperature(300.0) # K
seven.SetPressure(101325.0) # Pa
seven.SetMassFlow(1.0) # kg/s

'7: mass flow set to 1 kg/s'

In [10]:
# property package
Thermo_Package = sim.CreateAndAddPropertyPackage("Steam Tables (IAPWS-IF97)")

In [11]:
# Calc Modes
HT_1.CalcMode = HT_1.CalcMode.HeatAdded
HT_2.CalcMode = HT_2.CalcMode.OutletTemperature
HT_3.CalcMode = HT_3.CalcMode.OutletVaporFraction
HT_4.CalcMode = HT_4.CalcMode.TemperatureChange
# Setting Efficiencies
HT_1.set_Eficiencia(80)
HT_2.set_Eficiencia(70)
HT_3.set_Eficiencia(76)
HT_4.set_Eficiencia(85)
# Setting Properties
HT_1.DeltaQ = 200
HT_2.set_OutletTemperature(350)
HT_3.set_OutletVaporFraction(0.75)
HT_4.set_DeltaT(50)

In [12]:
# request a calculation
errors = interf.CalculateFlowsheet4(sim)
list(errors)

[]

In [13]:
# save file

fileNameToSave = Path.Combine(Environment.GetFolderPath(Environment.SpecialFolder.Desktop), "/workspace/00 FlowSheet Automation/04 Heater Automation/04 Heater.dwxmz")

interf.SaveFlowsheet(sim, fileNameToSave, True)